In [1]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copy

In [2]:
local_zip = 'D:\\Downloads\\kagglecatsanddogs_3367a.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('D:\\Other\\datasets')
zip_ref.close()

In [4]:
ds_loc = 'D:\\Other\\datasets'
print(len(os.listdir(ds_loc+'\\PetImages\\Cat\\')))
print(len(os.listdir(ds_loc+'\\PetImages\\Dog\\')))

12501
12501


In [8]:
os.mkdir(ds_loc+"\\PetImages\\train")
os.mkdir(ds_loc+"\\PetImages\\test")

for class_name in ['cats','dogs']:
    os.mkdir(ds_loc+"\\PetImages\\train\\"+class_name)
    os.mkdir(ds_loc+"\\PetImages\\test\\"+class_name)

In [10]:
from random import shuffle


def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):

    im_names = [i for i in os.listdir(SOURCE) if i[-1]=='g'\
                and os.path.getsize(SOURCE+"\\"+i)!=0]
    shuffle(im_names)

    for im in im_names[:int(len(im_names)*SPLIT_SIZE)]:
        copy(SOURCE+"\\"+im, TRAINING+"\\"+im)

    for im in im_names[int(len(im_names)*SPLIT_SIZE):]:
        copy(SOURCE+"\\"+im, TESTING+"\\"+im)



CAT_SOURCE = ds_loc + "\\PetImages\\Cat"
TRAINING_CATS = ds_loc + "\\PetImages\\train\\cats"
TESTING_CATS = ds_loc + "\\PetImages\\test\\cats"
DOG_SOURCE = ds_loc + "\\PetImages\\Dog"
TRAINING_DOGS = ds_loc + "\\PetImages\\train\\dogs"
TESTING_DOGS = ds_loc + "\\PetImages\\test\\dogs"

split_size = 0.9
split_data(CAT_SOURCE, TRAINING_CATS, TESTING_CATS, split_size)
split_data(DOG_SOURCE, TRAINING_DOGS, TESTING_DOGS, split_size)

In [11]:
print(len(os.listdir(TRAINING_CATS)))
print(len(os.listdir(TESTING_CATS)))
print(len(os.listdir(TRAINING_DOGS)))
print(len(os.listdir(TESTING_DOGS)))

11249
1250
11249
1250


In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, (3,3), input_shape=(150,150,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation=tf.nn.relu),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=['accuracy']
)

In [18]:
TRAINING_DIR = ds_loc+"\\PetImages\\train"
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    target_size=(150,150),
                                                    class_mode='binary')

TESTING_DIR = ds_loc+"\\PetImages\\test"
test_datagen = ImageDataGenerator(rescale=1/255)
test_generator = test_datagen.flow_from_directory(TESTING_DIR,
                                                    target_size=(150,150),
                                                    class_mode='binary')

Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [22]:
history = model.fit(train_generator, epochs=5,
                    verbose=1, validation_data=test_generator)



Epoch 1/5
 49/704 [=>............................] - ETA: 3:36 - loss: 0.5820 - accuracy: 0.6964

d:\users\i\pycharmprojects\pythonproject\venv\lib\site-packages\PIL\TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


704/704 [==============================] - 259s 367ms/step - loss: 0.5510 - accuracy: 0.7299 - val_loss: 0.5218 - val_accuracy: 0.7336

In [36]:
import numpy as np
from keras.preprocessing import image

path = "D:\\Downloads\\dog.jpg"
img = image.load_img(path, target_size=(150,150))
x1= image.img_to_array(img)
x1 = np.expand_dims(x1, axis=0)

path = "D:\\Downloads\\cat.jpg"
img = image.load_img(path, target_size=(150,150))
x2 = image.img_to_array(img)
x2 = tf.expand_dims(x2, axis=0)

x = np.vstack([x1,x2])
classes = model.predict(x)
print(classes)


[[1.]
 [0.]]
